<a href="https://colab.research.google.com/github/nadiduno/linkColabGoogle/blob/main/analise_de_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este código fornece uma abordagem básica para análise de churn em dados de um hospital, incluindo a geração de dados fictícios, pré-processamento dos dados, treinamento de um modelo de classificação e avaliação do desempenho do modelo.

**Explicação do Código:**
- O código começa gerando dados fictícios para um hospital, incluindo idade, sexo, número de consultas, número de exames, valor da fatura e churn (se o cliente deixou de utilizar o serviço).
- Em seguida, os dados são divididos em features (X) e target (y), e em conjunto de treinamento e teste.
- As features são pré-processadas usando `StandardScaler` para as features numéricas e `OneHotEncoder` para as features categóricas.
- Um modelo de classificação `RandomForestClassifier` é treinado com 100 estimadores.
- O desempenho do modelo é avaliado usando o relatório de classificação e a matriz de confusão.
- Finalmente, a importância das features é calculada e exibida.

**Explicação do resultado:**

**Relatório de Classificação:**
- Precision: A precisão é a proporção de verdadeiros positivos (casos positivos corretamente identificados) em relação a todos os casos positivos previstos pelo modelo. No caso da classe 0 (clientes que não churn), a precisão é 0.80, o que significa que 80% dos casos previstos como não churn foram corretos. Para a classe 1 (clientes que churn), a precisão é 0.00, o que significa que nenhum caso previsto como churn foi correto.
- Recall: O recall (ou sensibilidade) é a proporção de verdadeiros positivos em relação a todos os casos positivos reais. No caso da classe 0, o recall é 0.98, o que significa que o modelo identificou corretamente 98% dos casos de clientes que não churn. Para a classe 1, o recall é 0.00, o que significa que o modelo não identificou corretamente nenhum dos casos de clientes que churn.
- F1-score: O F1-score é uma média harmônica da precisão e do recall e é útil quando há um desequilíbrio de classes. Ele fornece uma única métrica que combina precisão e recall. No caso da classe 0, o F1-score é 0.88. Para a classe 1, o F1-score é 0.00.
- Support: O suporte é o número de ocorrências de cada classe no conjunto de teste.

**Matriz de Confusão:**
- A matriz de confusão mostra as classificações corretas e incorretas feitas pelo modelo. No caso da matriz de confusão fornecida, temos:
   - 158 verdadeiros negativos (TN): clientes que não churn, classificados corretamente como não churn.
   - 3 falsos positivos (FP): clientes que não churn, classificados incorretamente como churn.
   - 0 verdadeiros positivos (TP): clientes que churn, classificados corretamente como churn.
   - 39 falsos negativos (FN): clientes que churn, classificados incorretamente como não churn.

**Importância das Features:**
- A importância das features mostra a contribuição relativa de cada feature para a capacidade de prever o churn dos clientes. No caso fornecido, as features mais importantes são 'Valor da fatura', 'Idade' e 'Número de consultas', enquanto 'Sexo' tem uma importância relativamente baixa.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Gerar dados fictícios do hospital
import numpy as np

# Definir número de pacientes
num_pacientes = 1000

# Criar dados fictícios
np.random.seed(42)
idade = np.random.randint(18, 90, num_pacientes)
sexo = np.random.choice(['Masculino', 'Feminino'], num_pacientes)
num_consultas = np.random.randint(1, 10, num_pacientes)
num_exames = np.random.randint(1, 5, num_pacientes)
valor_fatura = np.random.randint(100, 10000, num_pacientes)
churn = np.random.choice([0, 1], num_pacientes, p=[0.8, 0.2])

# Criar DataFrame
dados_hospital = pd.DataFrame({
    'Idade': idade,
    'Sexo': sexo,
    'Número de consultas': num_consultas,
    'Número de exames': num_exames,
    'Valor da fatura': valor_fatura,
    'Churn': churn
})

# Salvar DataFrame em um arquivo CSV
dados_hospital.to_csv('dados_hospital.csv', index=False)

# Carregar os dados do hospital
dados_hospital = pd.read_csv('dados_hospital.csv')

# Dividir os dados em features (variáveis independentes) e target (variável dependente)
X = dados_hospital.drop('Churn', axis=1)
y = dados_hospital['Churn']

# Dividir os dados em conjunto de treinamento e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento das features
numeric_features = ['Idade', 'Número de consultas', 'Número de exames', 'Valor da fatura']
numeric_transformer = StandardScaler()

categorical_features = ['Sexo']
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Treinar um modelo de classificação (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o desempenho do modelo
y_pred = model.predict(X_test)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Identificar as features mais importantes para o modelo
feature_names = numeric_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
print("\nImportância das Features:")
print(feature_importances)



Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.80      0.98      0.88       161
           1       0.00      0.00      0.00        39

    accuracy                           0.79       200
   macro avg       0.40      0.49      0.44       200
weighted avg       0.65      0.79      0.71       200


Matriz de Confusão:
[[158   3]
 [ 39   0]]

Importância das Features:
               feature  importance
3      Valor da fatura    0.400081
0                Idade    0.311983
1  Número de consultas    0.167248
2     Número de exames    0.084492
4        Sexo_Feminino    0.019095
5       Sexo_Masculino    0.017100
